In [1]:
import torch
import retro
import gym
import torch
import torch.nn as nn

# import time to slow down the game
import time


# import env baseclass
from gym import Env
# import space shapes for the 
from gym.spaces import MultiBinary, Box
# import numpy to calculate change
import numpy as np
# for grayscale
import cv2
# for plotting
import matplotlib.pyplot as plt

# train and tune and the same time
# optimization framework
import optuna
import tqdm as notebook_tqdm
from stable_baselines3 import PPO, DDPG
from stable_baselines3.common.evaluation import evaluate_policy

# import sb3 monitor for logging
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, SubprocVecEnv
import os


import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import BasePolicy
from typing import Tuple

# game='SuperMarioWorld-Snes'


from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import VecMonitor
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
import os

import gym
import numpy as np
# from RandomAgent import TimeLimitWrapper

In [4]:
# env = retro.make(game='SuperMarioBros-Nes')

In [3]:
env.observation_space.shape

NameError: name 'env' is not defined

In [6]:
env.action_space

MultiBinary(9)

# Train

In [2]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq:
    :param log_dir: Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: Verbosity level.
    """
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), 'timesteps')
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose > 0:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose > 0:
                    print(f"Saving new best model to {self.save_path}")
                  self.model.save(self.save_path)

        return True

In [11]:
def make_env(env_id,rank,seed=0):
    def _init():
        env= retro.make(game=env_id)
        env = MaxAndSkipEnv(env,4)
        env.seed(seed+rank)
        return env
    
    set_random_seed(seed)
    return _init

log_dir="marioVITLog/"
os.makedirs(log_dir,exist_ok=True)

In [4]:
env_id = "SuperMarioBros-Nes"

In [5]:
num_cpu = 4

In [12]:
env = VecMonitor(SubprocVecEnv([make_env(env_id,i) for i in range(num_cpu)]))

In [19]:
model = PPO('MyPolicy',env,verbose=1,tensorboard_log='./boardVITMario/',learning_rate=0.00003)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [20]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [21]:
# callback = SaveOnBestTrainingRewardCallback(check_freq=1000,log_dir=log_dir)

In [22]:
CHECKPOINT_DIR = './mario_new/ViT/'

In [23]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)


In [24]:
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./boardVITMario/PPO_1
-----------------------------
| time/              |      |
|    fps             | 95   |
|    iterations      | 1    |
|    time_elapsed    | 85   |
|    total_timesteps | 8192 |
-----------------------------


OutOfMemoryError: CUDA out of memory. Tried to allocate 520.00 MiB (GPU 0; 6.00 GiB total capacity; 4.88 GiB already allocated; 0 bytes free; 4.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.save(env_id)